In [ ]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Weather 추가변수
- 나중에 모델 돌릴때마다 df에 추가하는 것

In [ ]:
# Daily WEATHER data
w_daily = pd.read_csv("/kaggle/input/weather/daily.csv",encoding='euc-kr', sep=',')
w_daily.head(2)

In [ ]:
w_daily2 = w_daily.drop(columns='Location')
w_daily2.head(2)

In [ ]:
w_daily2['Time'] = pd.to_datetime(w_daily2.Time)
w_daily2 = w_daily2.set_index('Time')
w_daily2.tail(2)

In [ ]:
######################
# Monthly WEATHER data
w_Monthly = pd.read_csv("/kaggle/input/weather/monthly.csv",encoding='euc-kr', sep=',')
w_Monthly.head(2)

In [ ]:
w_MonthlyTemp = w_Monthly.drop(columns=['Location', 'MaxTemp','MinTemp'])
w_Monthly2 = w_MonthlyTemp.iloc[12:,:]
w_Monthly2.head(2)

In [ ]:
w_Monthly2['Mth'] = pd.DataFrame([8, 9, 10,11,12, 1,2,3,4,5,6,7,8,9,10,11]).values
w_Monthly2.head()

In [ ]:
w_Monthly2 = w_Monthly2.drop(columns='Time')
w_Monthly2

## 파생 및 dummy변수

In [ ]:
test = pd.read_csv("/kaggle/input/dataset/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [ ]:
test.index = pd.to_datetime(test.index)
test.index

In [ ]:
### resampling by Day
test_D = test.resample('D').sum()

In [ ]:
def Daily_dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
#     df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [ ]:
TE_D_time_vars = Daily_dateTimeVar(test_D)

In [ ]:
TE_D_time_vars.head(1)

## Make DUMMIES

In [ ]:
test_D.head(2)

In [ ]:
TE_D_time_vars.head(2)

In [ ]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [ ]:
TE_D_timeV_dum = dummies(TE_D_time_vars, ['season', 'dayClass'])

In [ ]:
TE_D_timeV_dum.head(2)

In [ ]:
TE_timeV_Daily_dum_wthr= pd.concat([TE_D_timeV_dum, w_daily2], axis=1,join='inner')

In [ ]:
TE_timeV_Daily_dum_wthr.head()

In [ ]:
w_Monthly2Head = w_Monthly2.head(11)

In [ ]:
TE_timeV_Monthly_dum_wthr= pd.merge(TE_D_timeV_dum, w_Monthly2Head, left_on = 'Mth', right_on='Mth')
TE_timeV_Monthly_dum_wthr.shape

In [ ]:
TE_timeV_Monthly_dum_wthr.index = pd.date_range(start = pd.datetime(2017, 8, 29),
                             end = pd.datetime(2018, 6, 30),
                             freq = "D")

In [ ]:
TE_timeV_Monthly_dum_wthr.head()

# Modeling

* test : TEST셋의 순수한 meterIDs Time series (NA값 채워짐) 
* TE_timeV_dum: TEST셋의 시간파생변수 + dummy variables 

In [ ]:
test_D.head(1)

In [ ]:
TE_timeV_Daily_dum_wthr.head(1)

**### 칼럼 10~50개 돌리기

In [ ]:
##################################################
#### 칼럼 바꾸기 ###################################

test2 = test_D.iloc[:,:50]
test2.head()

In [ ]:
submission_daily = pd.date_range(start = pd.datetime(2018, 7, 1),
                             end = pd.datetime(2018, 7, 10),
                             freq = "D")
submission_daily

In [ ]:
tmp_test = pd.DataFrame(index = submission_daily)
X_test_D = dummies(Daily_dateTimeVar(tmp_test), ['season', 'dayClass'])
X_test_D['season_1'] =0 ; X_test_D['season_3'] =0 ;X_test_D['season_4'] =0
X_test_D['dayClass_1'] =0 ; X_test_D['dayClass_2'] =0 
X_test_D = pd.concat([X_test_D, w_daily2], axis=1,join='inner')

In [ ]:
X_test_D.head(2)

In [ ]:
submission_monthly = pd.date_range(start = pd.datetime(2018, 7, 1),
                             end = pd.datetime(2018, 11, 30),
                             freq = "D")
submission_monthly

In [ ]:
tmp_test = pd.DataFrame(index = submission_monthly)

X_test_M = dummies(Daily_dateTimeVar(tmp_test), ['season', 'dayClass'])
X_test_M['season_1'] =0 ; X_test_M['season_3'] =0 ;X_test_M['season_4'] =0
X_test_M['dayClass_1'] =0 ; X_test_M['dayClass_2'] =0 
X_test_M.head(2)

In [ ]:
w_Monthly_T = w_Monthly2.tail(5)

In [ ]:
X_test_M = pd.merge(X_test_M, w_Monthly_T, left_on = 'Mth', right_on='Mth', how='left')

X_test_M = X_test_M.set_index(submission_monthly)
X_test_M.index.name = 'Time'
X_test_M.head()

In [ ]:
train = test2 ; mID= 'X303'

dataM = pd.concat([train.loc[:,[mID]], TE_timeV_Monthly_dum_wthr], axis=1)
dataM = dataM.rename(columns={mID:'y'})
y_trainM = dataM.dropna().y
X_trainM = dataM.dropna().drop(['y'], axis=1)        
X_trainM

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[10, 50, 100],
    }

def gsLGBR_Daily_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(10):
        a['X2018_7_'+str(i+1)+'_d']=[fcst[i]]
    return a

def gsLGBR_Monthly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    
    a['X2018_7_m'] = [np.sum(fcst[:31])] # 7월 
    a['X2018_8_m'] = [np.sum(fcst[31:62])] # 8월
    a['X2018_9_m'] = [np.sum(fcst[62:92])] # 9월
    a['X2018_10_m'] = [np.sum(fcst[92:123])] # 10월
    a['X2018_11_m'] = [np.sum(fcst[123:153])]
    
    return a
   
def test50_Daily_GS_LGBReg(train, X_test_D, X_test_M):
    daily = pd.DataFrame()
    monthly = pd.DataFrame()
    for mID in train.columns:
#         # Day
#         dataD = pd.concat([train.loc[:,[mID]], TE_timeV_Daily_dum_wthr], axis=1)
#         dataD = dataD.rename(columns={mID:'y'})
#         for i in [7]:
#             dataD['lag_{}'.format(i)] = dataD.y.shift(i)
#         y_trainD = dataD.dropna().y
#         X_trainD = dataD.dropna().drop(['y'], axis=1)
        
#         X_test_D['lag_7'] = np.append(y_train[292:299].values, y_trainD[289:292].values)
#         model = LGBMRegressor(verbose=1, tree_learner='feature')
#         tmpD = gsLGBR_Daily_FitPred(model, lg_param,
#                                     mID, X_trainD, y_trainD, X_test_D)
#         daily = pd.concat([daily, tmpD], axis=0)
        
        ## Month
        dataM = pd.concat([train.loc[:,[mID]], TE_timeV_Monthly_dum_wthr], axis=1)
        dataM = dataM.rename(columns={mID:'y'})
        y_trainM = dataM.dropna().y
        X_trainM = dataM.dropna().drop(['y'], axis=1)        
        
        model = LGBMRegressor(verbose=1, tree_learner='feature')
        tmpM = gsLGBR_Monthly_FitPred(model, lg_param,
                            mID,X_trainM, y_trainM,X_test_M)
        monthly = pd.concat([monthly, tmpM], axis=0)
        
    return  daily, monthly

In [ ]:
test2.shape

In [ ]:
pred_D100, pred_M100 = test50_Daily_GS_LGBReg(test2, X_test_D,X_test_M)

In [ ]:
pred_D100.head()

In [ ]:
pred_M100.head()

In [ ]:
import os
pred_D100.to_csv(r'100_daily.csv')
pred_M100.to_csv(r'100_Monthy.csv')

from IPython.display import FileLink
FileLink(r'100_daily.csv')
FileLink(r'100_Monthy.csv')